# Первая часть задания - предобработка другого [датасета](https://www.kaggle.com/datasets/nehalbirla/vehicle-dataset-from-cardekho)

в архиве 4 файла с похожими датасета, отличаются сложность предобработки (больше признаков, столбцов) и кол-вом строк, выбрал самый сложный на мой взгляд датасет.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler

In [ ]:
df = pd.read_csv('https://drive.google.com/uc?id=17qBgxWuH-lgDuLrr3ih1BmIrW1rE3Cpv')
df


In [ ]:
print(df.shape)
print(df.dtypes)

In [ ]:
    df.info()

In [ ]:
df.describe()

## Предобработка


In [ ]:
df.dropna(inplace=True)
df = df.drop_duplicates()
df

только в одном месте сделал нормирование, т.к. для линейной регрессии она особо не нужна, просто показать и такую вещь:)


In [ ]:
df['year'] = (df['year'] - df['year'].min()) / (df['year'].max() - df['year'].min()) # нормирование соот.
mapping0 = {value: index for index, value in enumerate(df['fuel'].unique())}
df['fuel'] = df['fuel'].map(mapping0)
mapping1 = {value: index for index, value in enumerate(df['seller_type'].unique())}
df['seller_type'] = df['seller_type'].map(mapping1)
mapping2 = {value: index for index, value in enumerate(df['transmission'].unique())}
df['transmission'] = df['transmission'].map(mapping2)

In [ ]:
owner_mapping = {'First': 1,
                'Second': 2,
                'Third': 3,
                'Fourth': 4}
def replace_owner(owner):
    for key, value in owner_mapping.items():
        if key in owner:
            return value
    return None
df['owner'] = df['owner'].apply(replace_owner)

In [ ]:
def first_part(mileage):
    return str(mileage).split()[0]
name = df['name']
df['name'] = df['name'].apply(first_part)
mapping4 = {value: index for index, value in enumerate(df['name'].unique())}
df['name'] = df['name'].map(mapping4)
df['mileage'] = df['mileage'].apply(first_part).astype(float)
df['engine'] = df['engine'].apply(first_part).astype(float)
df['max_power'] = df['max_power'].apply(first_part).astype(float)
df = df.drop(['torque'], axis = 1)


df

здесь была попытка разбить столбец torque на два других с двумя соот. значениями, но моя реализация не совсем получилась из-за того, что в ячейке столбца может существовать 3 слова (реализовано только для 2х), пробелы в случайных местах, поэтому я отбросил этот признак (1 ячейка ниже просто существует)

In [ ]:
#def first_numb(parts):
    #for i in range(len(parts) - 1, -1, -1):
        #if parts[i].isdigit() != 1:
            #continue
        #else:
            #return parts[0:i + 1]

# Создаем функцию для обработки значений в ячейке
#def process_torque(torque):
    # Разделяем строку на части по пробелу
    #parts = torque.split()

    #first_num = first_numb(parts[0])
    # Извлекаем второе число или вычисляем среднее, если есть диапазон
    #if '-' in parts[1]:
        #parts = first_numb(parts[1])
        #num1, num2 = parts.split('-')
        #second_num = (int(num1) + int(num2)) / 2
    #else:
        #second_num = first_numb(parts[1])

    #return first_num, second_num
# Применяем функцию для обработки значений в колонке 'torque'
#df[['torque1', 'torque2']] = df['torque'].apply(process_torque)
#df

In [ ]:
df.info()

In [ ]:
df.dropna(inplace=True)
df

## Линейная регрессия

In [ ]:
Y = df['selling_price']
X = df.drop(['selling_price'], axis = 1)
X['abs term'] = [1 for i in range(len(df))]
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=100)
X_train

In [ ]:
A = X_train.values
A = np.matrix(A)
A_plus = np.linalg.inv((A.T @ A)) @ A.T
print('Псевдообратная матрица:\n', A_plus, '\n')
X = A_plus @ Y_train
print('Коэффициенты линейной регресии:\n', X)

In [ ]:
pred = X_test @ X.T
compare = pd.DataFrame({'fare':Y_test,'predictions':pred})
compare

In [ ]:
print('MAE = ', mean_absolute_error(Y_test,pred))
print('MSE = ', mean_squared_error(Y_test,pred))

In [ ]:
plt.figure(figsize=(16, 6))
plt.xlabel('Comparison between Real and Predicted values ')
sns.kdeplot(compare['fare'], fill=True, color='blue')
sns.kdeplot(compare['predictions'], color='red', fill=True)
plt.legend(labels = ['fare','predicted'])
plt.show()

# Вторая часть - полная реализация предсказания цен на такси [(датасет)](https://www.kaggle.com/datasets/raviiloveyou/predict-taxi-fare-with-a-bigquery-ml-forecasting) при помощи линейной регрессии и псевдообратной матрицы

In [ ]:
# т.к. датасет был уже разделен, а нас это не устраивает, то склеим два датасета
df = pd.read_csv('https://drive.google.com/uc?id=1LGVvuaeCq10xqn2xyKELooSiYKrYg2hW')
dff = pd.read_csv('https://drive.google.com/uc?id=1adTgj6Yh8eHvMgj5i3DZNE3i2v5b5CUl')
df = pd.concat([df, dff])
df

In [ ]:
print(df.shape)
print(df.dtypes)

## Предобработка

In [ ]:
df = df.drop_duplicates() # убрали дубликаты
df

In [ ]:
selected_columns = ['trip_duration', 'distance_traveled', 'num_of_passengers', 'fare', 'total_fare']
df = df[(df[selected_columns] > 0).all(axis=1)] # убрали нули, где они быть не могут
df

In [ ]:
df = df[~((df['trip_duration'] > 36000) & (df['distance_traveled'] < 10))]
df = df[(df['trip_duration'] < 72000)]
df = df[(df['distance_traveled'] < 1000)]
df = df[(df['tip'] < 500)]
# df = df[(df['num_of_passengers'] < 5)]

df

X - DataFrame с замененным столбцом, который мы хотим предсказать, на столбец, который в будущем будет отвечать за свободный коэфициент линейной регрессии

Y - столбец, который мы хотим предсказать

In [ ]:
Y = df['total_fare']
X = df.drop(['total_fare', 'fare'], axis = 1)
X['abs term'] = [1 for i in range(len(df))]
X

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=100)
X_train

In [ ]:
A = X_train.values
A = np.matrix(A)
A_plus = np.linalg.inv((A.T @ A)) @ A.T
print('Псевдообратная матрица:\n', A_plus, '\n')
X = A_plus @ Y_train
print('Коэффициенты линейной регресии:\n', X)

In [ ]:
pred = X_test @ X.T
compare = pd.DataFrame({'fare':Y_test,'predictions':pred})
compare

In [ ]:
print('MAE = ', mean_absolute_error(Y_test,pred))
print('MSE = ', mean_squared_error(Y_test,pred))

In [ ]:
plt.figure(figsize=(16, 6))
plt.xlabel('Comparison between Real and Predicted values ')
sns.kdeplot(compare['fare'], fill=True, color='blue')
sns.kdeplot(compare['predictions'], color='red', fill=True)
plt.legend(labels = ['fare','predicted'])
plt.show()

In [ ]:
lr = LinearRegression()
lr.fit(X_train,Y_train)
pred = lr.predict(X_test)
compare = pd.DataFrame({'Performance Index':Y_test,'Predictions':pred})
compare

In [ ]:
print('MAE = ', mean_absolute_error(Y_test,pred))
print('MSE = ', mean_squared_error(Y_test,pred))

Сошлось с готовй Lr ура!
